In [1]:
from sae_lens import SAE  # or whatever library
import torch




In [2]:
import json

# Load the steering vector weights from JSON
with open("../../directions/llama3.1_8b_base_instruct_directions/mms_balanced_shared.json", "r") as f:
    directions = json.load(f)


In [5]:
steering_vec = torch.tensor(directions['shared_directions']['21'])

# Load SAE for layer 21
sae = SAE.from_pretrained("fnlp/Llama-Scope", layer=21)  # pseudo-code

# SAE decoder: (num_features, hidden_dim)
decoder = sae.W_dec  

# Cosine similarity with each feature
steering_normed = steering_vec / steering_vec.norm()
decoder_normed = decoder / decoder.norm(dim=1, keepdim=True)
similarities = decoder_normed @ steering_normed

# Top aligned features
top_k = similarities.topk(20)
print(top_k.indices)  # feature IDs
print(top_k.values)   # how aligned

TypeError: SAE.from_pretrained() got an unexpected keyword argument 'layer'